# Preparing the Message Schedule

In [57]:
# Imports
from tutorial_functions import *

### Recap

Our initial message:

In [58]:
message = "This is a long message that is going to be two 512-bit blocks long."
message

'This is a long message that is going to be two 512-bit blocks long.'

We have our message converted from text to ascii to binary:

In [59]:
binary_message = ''.join(format(ord(char), '08b') for char in message)
binary_message

'01010100011010000110100101110011001000000110100101110011001000000110000100100000011011000110111101101110011001110010000001101101011001010111001101110011011000010110011101100101001000000111010001101000011000010111010000100000011010010111001100100000011001110110111101101001011011100110011100100000011101000110111100100000011000100110010100100000011101000111011101101111001000000011010100110001001100100010110101100010011010010111010000100000011000100110110001101111011000110110101101110011001000000110110001101111011011100110011100101110'

We have our padded message.

In [60]:
def pad_binary(binary_message,
               block_size=512,
               length_field=64):

    # Add the '1' bit
    padded = binary_message + '1'

    # Calculate Zeros Needed
    msg_size_on_final_block = (len(padded) % block_size)
    zeros_needed = block_size - msg_size_on_final_block - length_field

    # If there are not enough zeros, add another block w/ zeros
    if zeros_needed < 0:
        zeros_needed += 512
    
    # Otherwise, append the zeros needed
    padded += '0' * zeros_needed

    # Add 64-bit message length to the end, padded with zeros
    msg_length = len(binary_message)
    length_bits = format(msg_length, '064b')
    final_padded = padded + length_bits

    return final_padded

padded_message = pad_binary(binary_message)
padded_message

'010101000110100001101001011100110010000001101001011100110010000001100001001000000110110001101111011011100110011100100000011011010110010101110011011100110110000101100111011001010010000001110100011010000110000101110100001000000110100101110011001000000110011101101111011010010110111001100111001000000111010001101111001000000110001001100101001000000111010001110111011011110010000000110101001100010011001000101101011000100110100101110100001000000110001001101100011011110110001101101011011100110010000001101100011011110110111001100111001011101000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

SHA-256 works on 512 blocks. We divide the message into several blocks.

In [61]:
print(f"Padded Message Length: {len(padded_message)}")
print(f"Num Blocks Required: {len(padded_message) / 512}")

Padded Message Length: 1024
Num Blocks Required: 2.0


In [62]:
def divide_into_blocks(padded_message, block_size=512):
    # Ensure the padded message length is a multiple of block_size
    assert len(padded_message) % block_size == 0, "Padded message length must be a multiple of block size"
    
    # Split the padded message into blocks
    blocks = [padded_message[i:i + block_size] for i in range(0, len(padded_message), block_size)]
    
    return blocks

def print_block(block):
    # Print the block in a formatted manner
    for i in range(0, len(block), 32):
        print(block[i:i + 32])

blocks = divide_into_blocks(padded_message)
for i, block in enumerate(blocks):
    print(f"BLOCK {i + 1}:")
    print_block(block)
    print()

BLOCK 1:
01010100011010000110100101110011
00100000011010010111001100100000
01100001001000000110110001101111
01101110011001110010000001101101
01100101011100110111001101100001
01100111011001010010000001110100
01101000011000010111010000100000
01101001011100110010000001100111
01101111011010010110111001100111
00100000011101000110111100100000
01100010011001010010000001110100
01110111011011110010000000110101
00110001001100100010110101100010
01101001011101000010000001100010
01101100011011110110001101101011
01110011001000000110110001101111

BLOCK 2:
01101110011001110010111010000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
000000000000000000000000

We have our H Values

In [63]:
# Initial hash values
H = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

We have our K Constants

In [64]:
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

## Prepare Message Schedule

First, we're only going to focus on the first block of the message. We will build a function to iterate through all blocks later.

In [65]:
block = blocks[0]

In [66]:
W = [0] * 64
print_list_in_rows(W, 16)

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


### Populate the First 16 Words

Now we're going to fill W[0] to W[15] with the first 16 words of W.

Remember 512-bits divided by 32 bits is 16 words.

In [67]:
# Fill the first 16 words of W
for i in range(16):
    W[i] = int(block[i*32:(i+1)*32], 2)

In [78]:
print("Updated W with first 16 words:")
print_list_in_rows(W, 16)

Updated W with first 16 words:
1416128883, 543781664, 1629514863, 1852252269, 1702064993, 1734680692, 1751217184, 1769152615, 1869180519, 544501536, 1650794612, 2003771445, 825372002, 1769218146, 1819239275, 1931504751
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


Notice how the first word of the message schedule W[0] is the exact same as the first 32 bits of the block.

In [79]:
# Extract the first 32 bits of the block
first_32_bits_of_block = block[:32]

# Convert the first 32 bits of the block to an integer
first_32_bits_int = int(first_32_bits_of_block, 2)

# Compare with W[0]
print(f"First 32 bits of block (binary): {first_32_bits_of_block}")
print(f"First 32 bits of block (int): {first_32_bits_int}")
print(f"W[0] (int): {W[0]}")
print(f"Are they equal? {'Yes' if first_32_bits_int == W[0] else 'No'}")

First 32 bits of block (binary): 01010100011010000110100101110011
First 32 bits of block (int): 1416128883
W[0] (int): 1416128883
Are they equal? Yes


`right_rotate()`

For the remaining 48 words, they're going to be derived from a series of right rotations on the first 16 words.

In [81]:
def right_rotate(value, shift):
    return (value >> shift) | (value << (32 - shift)) & 0xFFFFFFFF

**Bitwise XOR (^)**
- Both the Same (0 and 0) or (1 + 1): 0
- Both Different (1 and 0) or (0 and 1): 1

`11001010`

`10111000`

`========`

`01110010`

In [87]:
# Define two binary numbers
a = 0b11001010
b = 0b10111000

# Perform XOR operation
result = a ^ b

# Print the binary representation of the numbers and the result
print(f"{a:08b} (a)")
print(f"{b:08b} (b)")
print(f"{result:08b} (a ^ b)")

11001010 (a)
10111000 (b)
01110010 (a ^ b)


Let's see how word 17 (`W[16]`) is created in this code.

**Step 1**
- `x0` = Take `W[0]` (first 32 bits of the block) and *right rotate* it by 7 bits.
- `y0` = Take `W[0]` again and *right rotate* it by 18 bits.
- `z0` = Take `W[0]` again and *right shift* it by 3 bits.
- `s0` = Do the XOR operations of `(x0 ^ y0) ^ z0`

**Step 2**
- `x1` = Take `W[15]` (last 32 bits of the block) and *right rotate* it by 17 bits.
- `y1` = Take `W[15]` again and *right rotate* it by 19 bits.
- `z1` = Take `W[15]` again and *right shift* it by 10 bits.
- `s1` = Do the XOR operations of `(x1 ^ y1) ^ z1`

**Step 3**
- To calculate the 17th word `W[16]`
    - Go 16 words back `W[0]`, without modifying it.
    - Add `s0`.
    - Add the word 7 words back `W[9]`
    - Add `s1`
    - Take that final number, and mask it with `0xFFFFFFFF` to ensure it fits in 32 bits.

##### NOTES
- The shifts and rotations of 7, 18, 3, 17, 19, and 10 bits are part of the SHA-256 algorithm.
- These numbers ensure that even a small change to the original message will result in a wildly different hash.
- The use of shifting means that some of the bits of the original are discarded, making it impossible to reverse engineer the original message.

**So what's happening here is lots of scrambling, and then scrambles of the scrambles or the scrambles, but it's done in a deterministic way so the same hash will always be reproduced. There's NO randomness.**


In [86]:
# Fill the remaining 48 words of W
for i in range(16, 64):
    # Calculate s0 using right rotations and right shift
    x0 = right_rotate(W[i-15], 7)
    y0 = right_rotate(W[i-15], 18)
    z0 = (W[i-15] >> 3)

    s0 =  x0 ^ y0 ^ z0 
    
    # Calculate s1 using right rotations and right shift
    x1 = right_rotate(W[i-2], 17)
    y1 = right_rotate(W[i-2], 19)
    z1 = (W[i-2] >> 10)
    s1 =  x1 ^ y1 ^ z1

    # Compute the i-th word in the message schedule
    W[i] = (W[i-16] + s0 + W[i-7] + s1) & 0xFFFFFFFF

# Print the complete W
print("Complete W:")
print_list_in_rows(W, 16)

Complete W:
1416128883, 543781664, 1629514863, 1852252269, 1702064993, 1734680692, 1751217184, 1769152615, 1869180519, 544501536, 1650794612, 2003771445, 825372002, 1769218146, 1819239275, 1931504751
3945172365, 2289223840, 2397601740, 4022102314, 2875422736, 2654289798, 3129462946, 626517073, 1775128597, 1788758459, 3762445189, 1507837325, 4158046136, 1106070233, 3849808407, 2397607552
2422983088, 1503464813, 3639923771, 4175367370, 790157116, 2440829288, 450286995, 4184850537, 1573971977, 3703860791, 589261572, 81660199, 3393539851, 207384749, 580786176, 607861249
3474241617, 3500651991, 4072462943, 939243534, 4146417512, 4012967680, 2319193487, 2397309598, 645548903, 715477939, 2496514163, 1556579279, 1581340539, 1245245934, 4212576149, 1173796238


In [94]:
[hex(w) for w in W]

['0x54686973',
 '0x20697320',
 '0x61206c6f',
 '0x6e67206d',
 '0x65737361',
 '0x67652074',
 '0x68617420',
 '0x69732067',
 '0x6f696e67',
 '0x20746f20',
 '0x62652074',
 '0x776f2035',
 '0x31322d62',
 '0x69742062',
 '0x6c6f636b',
 '0x73206c6f',
 '0xeb268d8d',
 '0x8872c8a0',
 '0x8ee87fcc',
 '0xefbc692a',
 '0xab637810',
 '0x9e353f86',
 '0xba87d0a2',
 '0x2557e451',
 '0x69ce5015',
 '0x6a9e49bb',
 '0xe0425b85',
 '0x59dfc58d',
 '0xf7d6bfb8',
 '0x41ed4ad9',
 '0xe5776a17',
 '0x8ee89680',
 '0x906bc9b0',
 '0x599d0d6d',
 '0xd8f4d43b',
 '0xf8df0cca',
 '0x2f18d73c',
 '0x917c1968',
 '0x1ad6d593',
 '0xf96fc069',
 '0x5dd0e809',
 '0xdcc46e37',
 '0x231f6b04',
 '0x4de0927',
 '0xca454f0b',
 '0xc5c70ad',
 '0x229e1800',
 '0x243b3a01',
 '0xcf14b851',
 '0xd0a7b5d7',
 '0xf2bcda5f',
 '0x37fbb80e',
 '0xf7254f68',
 '0xef310700',
 '0x8a3c158f',
 '0x8ee40a9e',
 '0x267a4b67',
 '0x2aa553b3',
 '0x94cdc873',
 '0x5cc783cf',
 '0x5e41577b',
 '0x4a38f1ee',
 '0xfb16cf95',
 '0x45f6b58e']


#### The Compression
The W message schedule is 64 words of 32 bits each. If we converted this to a hash right now, it would be 512 hexadecimal characters long. However, our goal is to produce a concise and fixed-length hash, typically 64 characters long, which is more practical for use in various applications like digital signatures, data integrity checks, and password storage. Therefore, we need to compress the message schedule down to a 64 character hash. This compression ensures that the hash is both manageable in size and consistent in length, regardless of the input message size.

Decoupling the H values into `a, b, c, d, e, f, g, h`

In [89]:
# Constants for SHA-256
H = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

# Initialize working variables with the current hash value
a, b, c, d, e, f, g, h = H

Now let's do the compression loop.

##### Multiple Rounds
`for i in range(64):`: We are going to repeat this process 64 times. Even though it would be extremely difficlt to reverse engineer the original message with one compression, by doing 64 rounds, we're making it impossible.

##### S1
- `x1` = right rotate the 5th hash `(e)` by **6** 
- `y1` = right rotate the 5th hash `(e)` by **11** 
- `x1` = right rotate the 5th hash `(e)` by **25** 
- `S1` = apply XOR operations (x1 ^ y1) ^ z1 

In [ ]:
# Main hash computation loop
for i in range(64):
    
    # compute S1
    x1 = right_rotate(e, 6)
    y1 = right_rotate(e, 11)
    z1 = right_rotate(e, 25)
    S1 =  x1 ^ y1 ^ z1


    ch = (e & f) ^ (~e & g)
    temp1 = (h + S1 + ch + K[i] + W[i]) & 0xFFFFFFFF
    S0 = right_rotate(a, 2) ^ right_rotate(a, 13) ^ right_rotate(a, 22)
    maj = (a & b) ^ (a & c) ^ (b & c)
    temp2 = (S0 + maj) & 0xFFFFFFFF

    h = g
    g = f
    f = e
    e = (d + temp1) & 0xFFFFFFFF
    d = c
    c = b
    b = a
    a = (temp1 + temp2) & 0xFFFFFFFF

# Add the compressed chunk to the current hash value
H = [(x + y) & 0xFFFFFFFF for x, y in zip(H, [a, b, c, d, e, f, g, h])]

# Print the resulting hash
print("Resulting Hash:")
print(''.join(f'{value:08x}' for value in H))